# Sarcasm Detector

## Get and Load Data

In [1]:
!pip install --upgrade --no-cache-dir gdown

In [3]:
!gdown --id 1ytPDo88FEC2ArOjdqErAiarAZBNJzEJz

C:\Users\aster\apps\anaconda3\Lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ytPDo88FEC2ArOjdqErAiarAZBNJzEJz
To: C:\Users\aster\dev\git\hub\migros-construct-dc\fake-news-detector\notebook\SarcasmDetect.json

  0%|          | 0.00/6.06M [00:00<?, ?B/s]
  9%|8         | 524k/6.06M [00:00<00:01, 3.37MB/s]
 17%|#7        | 1.05M/6.06M [00:00<00:01, 3.17MB/s]
 26%|##5       | 1.57M/6.06M [00:00<00:01, 3.81MB/s]
 35%|###4      | 2.10M/6.06M [00:00<00:00, 3.99MB/s]
 43%|####3     | 2.62M/6.06M [00:00<00:00, 4.11MB/s]
 52%|#####1    | 3.15M/6.06M [00:00<00:00, 4.04MB/s]
 61%|######    | 3.67M/6.06M [00:00<00:00, 4.21MB/s]
 69%|######9   | 4.19M/6.06M [00:01<00:00, 3.67MB/s]
 78%|#######7  | 4.72M/6.06M [00:01<00:00, 3.97MB/s]
 87%|########6 | 5.24M/6.06M [00:01<00:00, 3.97MB/s]
 95%|########

In [4]:
import pandas as pd

df = pd.read_json('./SarcasmDetect.json', lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


## Remove all records with no headline text

In [5]:
df = df[df['headline'] != '']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


Lets start with the key objectives

# Key Objectives

Train-Test Split should be 70:30, use random_state=42 for same data split throughout. Drop the article_link column as it can be used as a leakage to identify fake news directly (theonion makes only fake news)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset
X = df['headline']
y = df['is_sarcastic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [2]:
!pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/76/4f/39ddae9fb07b8c039fa5a5f2b6623c6e0564199d82da33fcef62bcf93174/tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/14/5a/0e2c734acb91d22fa67ccb7f0cc869e24c418486aaba3d7ca8cad158d5a0/tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2d

In [7]:
import tensorflow as tf
max_features = 10000  # Maximum vocab size
sequence_length = 250  # Maximum length of each headline

text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)


ModuleNotFoundError: No module named 'distutils'